In [8]:
from pydub import AudioSegment
from pydub.generators import Sine
import math

def hex_to_rgb(hex_color):
    hex_color = hex_color.lstrip('#')
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

def rgb_to_hex(rgb):
    """Convert an RGB color to hexadecimal format."""
    return '#{:02x}{:02x}{:02x}'.format(*rgb)

def color_distance(rgb1, rgb2):
    """Calculate the Euclidean distance between two RGB colors."""
    return math.sqrt(sum((c1 - c2) ** 2 for c1, c2 in zip(rgb1, rgb2)))

def closest_color(rgb, color_map):
    """Find the closest color in the color_map to the given RGB color."""
    closest_colors = sorted(color_map.keys(), key=lambda color: color_distance(rgb, hex_to_rgb(color)))
    return closest_colors[0]


def image_to_music(main_colors, pixels):
    color_to_freq_map = {
        "#00290e": 261.63,  # C4
        "#fefefe": 293.66,  # D4
        '#d11033': 329.63,  # E4
    }

    song = AudioSegment.silent(duration=0)  # Start with a silent segment
    note_duration = 1000  # 1 second per note

    for row in pixels:
        row_length = len(row)
        color_counts = {}

        # Count occurrences of each color in the row
        for idx in row:
            hex_color = main_colors[idx]
            if hex_color in color_counts:
                color_counts[hex_color] += 1
            else:
                color_counts[hex_color] = 1

        # Generate a note for each color with its proportional duration
        for hex_color, count in color_counts.items():
            proportion = count / row_length
            note_time = int(note_duration * proportion)
            rgb_color = hex_to_rgb(hex_color)
            closest_hex = closest_color(rgb_color, color_to_freq_map)
            freq = color_to_freq_map[closest_hex]
            note = Sine(freq).to_audio_segment(duration=note_time)
            song += note


    output_file = "output_song.midi"
    song.export(output_file, format="midi")

    print(main_colors)
    print(pixels)

In [13]:
from PIL import Image
import numpy as np
from sklearn.cluster import KMeans

def load_and_cluster(path, n_clusters, display=False):
    image = Image.open(path)
    x, y = image.size
    
    h = 150 / float(y)

    redim = min(x, h)

    new_h = int(y * redim)

    image = image.resize((x, new_h), Image.ANTIALIAS)
    
    x, y = image.size

    if image.mode != 'RGB':
        image = image.convert('RGB')

    pixels = list(image.getdata())
    pixels_list = [pixels[i * x:(i + 1) * x] for i in range(y)]
    image_array = np.array(image)
    pixels = image_array.reshape(-1, 3)

    kmeans = KMeans(n_clusters=n_clusters)
    kmeans.fit(pixels)
    labels = kmeans.labels_

    for i in range(y):
        for j in range(x):
            pixel_index = i * x + j
            pixels_list[i][j] = labels[pixel_index]
    centers = [rgb_to_hex(i) for i in kmeans.cluster_centers_.astype(int)]
    return centers, pixels_list

In [14]:
main_colors, pixels_list = load_and_cluster("Flag_of_Sudan.png", 3)
print(main_colors)

['#fefefe', '#00290e', '#d11033']


In [ ]:
image_to_music(main_colors, pixels_list)

['#d11033', '#fefefe', '#00290e']
[[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0